In [1]:
import cv2

In [3]:
import os
import glob
import sys, argparse

In [4]:
import numpy as np
from scipy import optimize as opt

In [22]:
def get_camera_images():
    images = [each for each in glob.glob(IMG_DIR + "*.jpg")]
    images = sorted(images)
    for each in images:
        yield (each, cv2.imread(each, 0))
    # yield [(each, cv2.imread(each, 0)) for each in images]


In [34]:
IMG_DIR = '/home/seonwookim/Projects/GIST-ADIP/images/'
DEBUG_DIR = '/home/seonwookim/Projects/GIST-ADIP/images/Debug/'
PATTERN_SIZE = (6, 4)
SQUARE_SIZE = 1.0

In [37]:
def show_image(string, image):
    cv2.imshow(string, image)
    cv2.waitKey()

In [6]:
def getChessboardCorners(images = None, visualize=False):
    objp = np.zeros((PATTERN_SIZE[1]*PATTERN_SIZE[0], 3), dtype=np.float64)
    objp[:, :2] = np.indices(PATTERN_SIZE).T.reshape(-1, 2)
    objp *= SQUARE_SIZE

    chessboard_corners = []
    image_points = []
    object_points = []
    correspondences = []
    ctr=0
    for (path, each) in get_camera_images(): #images:
        print("Processing Image : ", path)
        ret, corners = cv2.findChessboardCorners(each, patternSize=PATTERN_SIZE)
        if ret:
            print ("Chessboard Detected ")
            corners = corners.reshape(-1, 2)            
            if corners.shape[0] == objp.shape[0] :
                image_points.append(corners)
                object_points.append(objp[:,:-1]) #append only World_X, World_Y. Because World_Z is ZERO. Just a simple modification for get_normalization_matrix
                assert corners.shape == objp[:, :-1].shape, "mismatch shape corners and objp[:,:-1]"
                correspondences.append([corners.astype(np.int), objp[:, :-1].astype(np.int)])
            if visualize:
                # Draw and display the corners
                ec = cv2.cvtColor(each, cv2.COLOR_GRAY2BGR)
                cv2.drawChessboardCorners(ec, PATTERN_SIZE, corners, ret)
                cv2.imwrite(DEBUG_DIR + str(ctr)+".png", ec)                
        else:
            print ("Error in detection points", ctr)

        ctr+=1

    return correspondences

In [39]:
def normalize_points(chessboard_correspondences):
    views = len(chessboard_correspondences)

    def get_normalization_matrix(pts, name="A"):
        pts = pts.astype(np.float64)
        x_mean, y_mean = np.mean(pts, axis=0)
        var_x, var_y = np.var(pts, axis=0)

        s_x , s_y = np.sqrt(2/var_x), np.sqrt(2/var_y)
        
        print("Matrix: {4} : meanx {0}, meany {1}, varx {2}, vary {3}, sx {5}, sy {6} ".format(x_mean, y_mean, var_x, var_y, name, s_x, s_y))

        n = np.array([[s_x, 0, -s_x*x_mean], [0, s_y, -s_y*y_mean], [0, 0, 1]])
        # print(n)

        n_inv = np.array([ [1./s_x ,  0 , x_mean], [0, 1./s_y, y_mean] , [0, 0, 1] ])
        return n.astype(np.float64), n_inv.astype(np.float64)


    ret_correspondences = [] 
    for i in range(views):
        imp, objp = chessboard_correspondences[i]
        N_x, N_x_inv = get_normalization_matrix(objp, "A")
        N_u, N_u_inv = get_normalization_matrix(imp, "B")
        # print(N_x)
        # print(N_u)
        # convert imp, objp to homogeneous
        # hom_imp = np.array([np.array([[each[0]], [each[1]], [1.0]]) for each in imp])
        # hom_objp = np.array([np.array([[each[0]], [each[1]], [1.0]]) for each in objp])
        hom_imp = np.array([ [[each[0]], [each[1]], [1.0]] for each in imp])
        hom_objp = np.array([ [[each[0]], [each[1]], [1.0]] for each in objp])


        normalized_hom_imp = hom_imp
        normalized_hom_objp = hom_objp

        for i in range(normalized_hom_objp.shape[0]):
            # 54 points. iterate one by onea
            # all points are homogeneous
            n_o = np.matmul(N_x,normalized_hom_objp[i])
            normalized_hom_objp[i] = n_o/n_o[-1]
            
            n_u = np.matmul(N_u,normalized_hom_imp[i])
            normalized_hom_imp[i] = n_u/n_u[-1]

        normalized_objp = normalized_hom_objp.reshape(normalized_hom_objp.shape[0], normalized_hom_objp.shape[1])
        normalized_imp = normalized_hom_imp.reshape(normalized_hom_imp.shape[0], normalized_hom_imp.shape[1])

        normalized_objp = normalized_objp[:,:-1]        
        normalized_imp = normalized_imp[:,:-1]

        # print(normalized_imp)

        ret_correspondences.append((imp, objp, normalized_imp, normalized_objp, N_u, N_x, N_u_inv, N_x_inv))

    return ret_correspondences

In [43]:
def compute_view_based_homography(correspondence, reproj = False):
    """
    correspondence = (imp, objp, normalized_imp, normalized_objp, N_u, N_x, N_u_inv, N_x_inv)
    """
    image_points = correspondence[0]
    object_points = correspondence[1]
    normalized_image_points = correspondence[2]
    normalized_object_points = correspondence[3]
    N_u = correspondence[4]
    N_x = correspondence[5]
    N_u_inv = correspondence[6]
    N_x_inv = correspondence[7]



    N = len(image_points)
    print("Number of points in current view : ", N)

    M = np.zeros((2*N, 9), dtype=np.float64)
    print("Shape of Matrix M : ", M.shape)

    print("N_model\n", N_x)
    print("N_observed\n", N_u)

    # create row wise allotment for each 0-2i rows
    # that means 2 rows.. 
    for i in range(N):
        X, Y = normalized_object_points[i] #A
        u, v = normalized_image_points[i] #B

        row_1 = np.array([ -X, -Y, -1, 0, 0, 0, X*u, Y*u, u])
        row_2 = np.array([ 0, 0, 0, -X, -Y, -1, X*v, Y*v, v])
        M[2*i] = row_1
        M[(2*i) + 1] = row_2

        print ("p_model {0} \t p_obs {1}".format((X, Y), (u, v)))



    # M.h  = 0 . solve system of linear equations using SVD
    u, s, vh = np.linalg.svd(M)
    print("Computing SVD of M")
    # print("U : Shape {0} : {1}".format(u.shape, u))
    # print("S : Shape {0} : {1}".format(s.shape, s))
    # print("V_t : Shape {0} : {1}".format(vh.shape, vh))
    # print(s, np.argmin(s))

    h_norm = vh[np.argmin(s)]
    h_norm = h_norm.reshape(3, 3)
    # print("Normalized Homography Matrix : \n" , h_norm)
    print(N_u_inv)
    print(N_x)
    # h = h_norm
    h = np.matmul(np.matmul(N_u_inv,h_norm), N_x)
    
    # if abs(h[2, 2]) > 10e-8:
    h = h[:,:]/h[2, 2]


    print("Homography for View : \n", h )

    if reproj:
        reproj_error = 0
        for i in range(len(image_points)):
            t1 = np.array([[object_points[i][0]], [object_points[i][1]], [1.0]])
            t = np.matmul(h, t1).reshape(1, 3)
            t = t/t[0][-1]
            formatstring = "Imp {0} | ObjP {1} | Tx {2}".format(image_points[i], object_points[i], t)
            print(formatstring)
            reproj_error += np.sum(np.abs(image_points[i] - t[0][:-1]))
        reproj_error = np.sqrt(reproj_error/N)/100.0
        print("Reprojection error : ", reproj_error)

    return h


In [47]:
def minimizer_func(initial_guess, X, Y, h, N):
    # X : normalized object points flattened
    # Y : normalized image points flattened
    # h : homography flattened
    # N : number of points
    # 
    x_j = X.reshape(N, 2)
    # Y = Y.reshape(N, 2)
    # h = h.reshape(3, 3)
    projected = [0 for i in range(2*N)]
    for j in range(N):
        x, y = x_j[j]
        w = h[6]*x + h[7]*y + h[8]
        # pts = np.matmul(np.array([ [h[0], h[1], h[2]] , [h[3], h[4], h[5]]]), np.array([ [x] , [y] , [1.]]))
        # pts = pts/float(w)
        # u, v = pts[0][0], pts[1][0]
        projected[2*j] = (h[0] * x + h[1] * y + h[2]) / w
        projected[2*j + 1] = (h[3] * x + h[4] * y + h[5]) / w

    # return projected
    return (np.abs(projected - Y))**2

In [10]:
def jac_function(initial_guess, X, Y, h, N):
    x_j = X.reshape(N, 2)
    jacobian = np.zeros( (2*N, 9) , np.float64)
    for j in range(N):
        x, y = x_j[j]
        sx = np.float64(h[0]*x + h[1]*y + h[2])
        sy = np.float64(h[3]*x + h[4]*y + h[5])
        w = np.float64(h[6]*x + h[7]*y + h[8])
        jacobian[2*j] = np.array([x/w, y/w, 1/w, 0, 0, 0, -sx*x/w**2, -sx*y/w**2, -sx/w**2])
        jacobian[2*j + 1] = np.array([0, 0, 0, x/w, y/w, 1/w, -sy*x/w**2, -sy*y/w**2, -sy/w**2])

    return jacobian

In [11]:
def refine_homographies(H, correspondences, skip=False):
    if skip:
        return H

    image_points = correspondence[0]
    object_points = correspondence[1]
    normalized_image_points = correspondence[2]
    normalized_object_points = correspondence[3]
    N_u = correspondence[4]
    N_x = correspondence[5]
    N_u_inv = correspondence[6]
    N_x_inv = correspondence[7]

    N = normalized_object_points.shape[0]
    X = object_points.flatten()
    Y = image_points.flatten()
    h = H.flatten()
    h_prime = opt.least_squares(fun=minimizer_func, x0=h, jac=jac_function, method="lm" , args=[X, Y, h, N], verbose=0)

    
    if h_prime.success:
        H =  h_prime.x.reshape(3, 3)
    H = H/H[2, 2]
    return H

In [12]:
def get_intrinsic_parameters(H_r):
    M = len(H_r)
    V = np.zeros((2*M, 6), np.float64)

    def v_pq(p, q, H):
        v = np.array([
                H[0, p]*H[0, q],
                H[0, p]*H[1, q] + H[1, p]*H[0, q],
                H[1, p]*H[1, q],
                H[2, p]*H[0, q] + H[0, p]*H[2, q],
                H[2, p]*H[1, q] + H[1, p]*H[2, q],
                H[2, p]*H[2, q]
            ])
        return v

    for i in range(M):
        H = H_r[i]
        V[2*i] = v_pq(p=0, q=1, H=H)
        V[2*i + 1] = np.subtract(v_pq(p=0, q=0, H=H), v_pq(p=1, q=1, H=H))

    # solve V.b = 0
    u, s, vh = np.linalg.svd(V)
    # print(u, "\n", s, "\n", vh)
    b = vh[np.argmin(s)]
    print("V.b = 0 Solution : ", b.shape)

    # according to zhangs method
    vc = (b[1]*b[3] - b[0]*b[4])/(b[0]*b[2] - b[1]**2)
    l = b[5] - (b[3]**2 + vc*(b[1]*b[2] - b[0]*b[4]))/b[0]
    alpha = np.sqrt((l/b[0]))
    beta = np.sqrt(((l*b[0])/(b[0]*b[2] - b[1]**2)))
    gamma = -1*((b[1])*(alpha**2) *(beta/l))
    uc = (gamma*vc/beta) - (b[3]*(alpha**2)/l)

    print([vc,
            l,
            alpha,
            beta,
            gamma,
        uc])

    A = np.array([
            [alpha, gamma, uc],
            [0, beta, vc],
            [0, 0, 1.0],
        ])
    print("Intrinsic Camera Matrix is :")
    print(A)
    return A


In [50]:
chessboard_correspondences = getChessboardCorners(images=None, visualize=True)

['/home/seonwookim/Projects/GIST-ADIP/images/cali11.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali6.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali2.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali7.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali10.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali4.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali5.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali3.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali1.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali9.jpg', '/home/seonwookim/Projects/GIST-ADIP/images/cali8.jpg']
Processing Image :  /home/seonwookim/Projects/GIST-ADIP/images/cali1.jpg
Chessboard Detected 
Processing Image :  /home/seonwookim/Projects/GIST-ADIP/images/cali10.jpg
Chessboard Detected 
Processing Image :  /home/seonwookim/Projects/GIST-ADIP/images/cali11.jpg
Chessboard Detected 
Processing Image :  /home/seonwookim/Projects/GIST-ADIP/images/cali2.jpg
Chessboard Detected 
Pro

In [51]:
chessboard_correspondences_normalized = normalize_points(chessboard_correspondences)

Matrix: A : meanx 2.5, meany 1.5, varx 2.9166666666666665, vary 1.25, sx 0.828078671210825, sy 1.2649110640673518 
Matrix: B : meanx 2147.5416666666665, meany 1522.5, varx 388315.08159722225, vary 164907.83333333334, sx 0.0022694617565975767, sy 0.003482525899838226 
Matrix: A : meanx 2.5, meany 1.5, varx 2.9166666666666665, vary 1.25, sx 0.828078671210825, sy 1.2649110640673518 
Matrix: B : meanx 1932.2083333333333, meany 1508.2916666666667, varx 271045.5815972222, vary 135335.62326388888, sx 0.0027164006879491884, sy 0.0038442261988902407 
Matrix: A : meanx 2.5, meany 1.5, varx 2.9166666666666665, vary 1.25, sx 0.828078671210825, sy 1.2649110640673518 
Matrix: B : meanx 2185.2916666666665, meany 1506.4583333333333, varx 156097.6232638889, vary 112879.66493055558, sx 0.00357945455165661, sy 0.004209273469283811 
Matrix: A : meanx 2.5, meany 1.5, varx 2.9166666666666665, vary 1.25, sx 0.828078671210825, sy 1.2649110640673518 
Matrix: B : meanx 1775.0416666666667, meany 1510.125, varx 3

In [52]:
print("M = ", len(chessboard_correspondences_normalized), " view images")
print("N = ", len(chessboard_correspondences_normalized[0][0]),  " points per image")

M =  11  view images
N =  24  points per image


In [53]:
H = []
for correspondence in chessboard_correspondences_normalized:
    H.append(compute_view_based_homography(correspondence, reproj=0))

Number of points in current view :  24
Shape of Matrix M :  (48, 9)
N_model
 [[ 0.82807867  0.         -2.07019668]
 [ 0.          1.26491106 -1.8973666 ]
 [ 0.          0.          1.        ]]
N_observed
 [[ 2.26946176e-03  0.00000000e+00 -4.87376368e+00]
 [ 0.00000000e+00  3.48252590e-03 -5.30214568e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
p_model (-2.0701966780270626, -1.8973665961010275) 	 p_obs (-2.098211954880984, -1.92409555966062)
p_model (-1.2421180068162376, -1.8973665961010275) 	 p_obs (-1.2494332579134908, -1.910165456061267)
p_model (-0.4140393356054126, -1.8973665961010275) 	 p_obs (-0.4097324079723874, -1.9032004042615909)
p_model (0.4140393356054122, -1.8973665961010275) 	 p_obs (0.42542951845552057, -1.9032004042615909)
p_model (1.2421180068162374, -1.8973665961010275) 	 p_obs (1.2719387536664168, -1.910165456061267)
p_model (2.0701966780270626, -1.8973665961010275) 	 p_obs (2.113909065364118, -1.910165456061267)
p_model (-2.0701966780270626, -0.632455

In [54]:
H_r = []
for i in range(len(H)):
    h_opt = refine_homographies(H[i], chessboard_correspondences_normalized[i], skip=False)
    H_r.append(h_opt)

In [55]:
A = get_intrinsic_parameters(H_r)

V.b = 0 Solution :  (6,)
[1495.1975588647115, 0.5766943102322538, 2889.7025478795254, 2888.609112558861, -2.051264303427118, 1971.2339095482118]
Intrinsic Camera Matrix is :
[[ 2.88970255e+03 -2.05126430e+00  1.97123391e+03]
 [ 0.00000000e+00  2.88860911e+03  1.49519756e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [56]:
H_r

[array([[3.70519824e+02, 1.92117175e+01, 1.22552571e+03],
        [4.42807187e-01, 3.84297351e+02, 9.73035738e+02],
        [4.35099295e-05, 1.01992776e-02, 1.00000000e+00]]),
 array([[ 2.07475592e+02,  3.87056466e+00,  1.21781151e+03],
        [-3.86117552e+01,  3.09658276e+02,  1.01854381e+03],
        [-3.67557261e-02,  7.02376439e-03,  1.00000000e+00]]),
 array([[4.22300801e+02, 2.68189945e+01, 1.54219237e+03],
        [8.10012061e+01, 3.62584693e+02, 1.03956945e+03],
        [6.90961341e-02, 8.80912647e-03, 1.00000000e+00]]),
 array([[ 3.19335465e+02,  2.37950153e+01,  8.62530673e+02],
        [-2.71391069e+01,  3.76713161e+02,  9.68619075e+02],
        [-1.94454723e-02,  1.07090477e-02,  1.00000000e+00]]),
 array([[ 2.18471613e+02,  1.33577070e+01,  1.14469165e+03],
        [-5.48253365e+01,  3.64010294e+02,  9.60959244e+02],
        [-5.01552841e-02,  1.08020678e-02,  1.00000000e+00]]),
 array([[5.10745990e+02, 2.81036426e+01, 1.35413029e+03],
        [8.51835056e+01, 4.42373964